# Cuaderno 4: Regla de Oja

En este cuaderno probaremos algunas reglas de plasticidad sináptica y sus consecuencias estadísticas.

## Configuración

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Funciones de graficado

In [ ]:
def visualizar_red(x, w):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), layout="constrained")
  ax1.plot(x, label=["Neurona pre-sináptica 1", "Neurona pre-sináptica 2"])
  ax1.plot(x @ w, label="Neurona post-sináptica")
  ax1.set_xlabel("Tiempo (s)")
  ax1.set_ylabel("Frecuencia (Hz)")
  ax1.legend()

  # Graficamos los puntos
  z = x - np.mean(x, axis=0)
  ax2.scatter(z[:, 0], z[:, 1])
  ax2.arrow(0, 0, w[0].item(), w[1].item(), width=0.01)
  ax2.set_xlabel("Neurona pre-sináptica 1")
  ax2.set_ylabel("Neurona pre-sináptica 2")

  plt.show()

def visualizar_vector(v):
  """ Dibuja un vector 2D

  Argumentos:
    v (ndarray): array de tamaño (2,) con las coordenadas del vector
  """
  fig, ax = plt.subplots()

  # Set up plot aesthetics
  ax.spines['top'].set_color('none')
  ax.spines['bottom'].set_position('zero')
  ax.spines['left'].set_position('zero')
  ax.spines['right'].set_color('none')
  ax.set(xlim = [-6, 6], ylim = [-6, 6])
  ax.grid(True, alpha=.4)

  # Plot vectors
  v_arr = ax.arrow(0, 0, v[0], v[1], color='black', width=0.08, length_includes_head=True)
  ax.set(xlim = [-4, 4], ylim = [-4, 4])

## Introducción a vectores

Consideremos primero un vector $\pmb{a}$ definido como:

$$ \pmb{a} = \begin{bmatrix} a_{1} \\ a_{2} \end{bmatrix} $$

Un vector se puede considerar desde al menos dos perspectivas: como una lista ordenada de números o como una flecha con la base en el origen de un sistema de coordenadas. Son dos maneras de mirar lo mismo: en el caso de la flecha, la punta está definida por una coordenada (que puede representarse con la lista ordenada).

La **dimensionalidad** de un vector está determinada por la cantidad de componentes en la lista ordenada (o por la dimensionalidad del espacio en el que existe la flecha). Decimos, entonces, que este vector de dos dimensiones tiene 2 componentes: $a_1$ y $a_2$.

Veamos como inicializar y visualizar un vector arbitrario en Numpy.

In [ ]:
a = np.array([2, 1])
print(a)

Cuando lo imprimimos, un vector no parece ser otra cosa que una lista ordenada. Generalmente, es mejor visualizarlo como una flecha en un espacio n-dimensional. Ejecutá la celda siguiente para visualizar este vector en un espacio de dos dimensiones:

In [ ]:
@widgets.interact(a1=widgets.IntSlider(2, -4, 4, description="$a_{1}$"), a2=widgets.IntSlider(1, -4, 4, description="$a_{2}$"))
def interactive(a1, a2=1):
  visualizar_vector(np.array([a1, a2]))

## Nuestra primera red neuronal

Definamos un modelo neuronal muy simple: un sistema de dos neuronas en el cual podemos medir su actividad. El vector $\pmb{x}$ define la actividad de las dos neuronas en un instante dado:

$$ \pmb{x} = \begin{bmatrix} 2 & 3 \end{bmatrix} $$

Tiene, entonces, dos componentes: $x_{1}$ y $x_{2}$. Uno para definir la actividad de cada neurona.

Agreguemos una tercera neurona, la neurona post-sináptica, cuya actividad esté mediada por la actividad de del sistema de neuronas recién definido. Para modelar la influencia que cada neurona pueda tener sobre esta tercera neurona, podemos definir un segundo vector $\pmb{w}$:

$$ \pmb{w} = \begin{bmatrix} 1 \\ 4 \end{bmatrix} $$

donde sus compontentes, $w_{1}$ y $w_{2}$, modelan el peso que tiene cada neurona $x_{1}$ y $x_{2}$ respectivamente sobre esta tercera neurona.

La actividad de esta tercera neurona se puede modelar como el **producto escalar** entre estos dos vectores:

$$ \pmb{x} \cdot \pmb{w} = x_{1} w_{1} + x_{2} w_{2} = 2 \times 1 + 3 \times 4 = 14 $$

Veamos como hacerlo en Python:

In [ ]:
x = np.array([2, 3])
w = np.array([1, 4])

print(x @ w)

Para modelar la actividad de las tres neuronas a lo largo del tiempo, podemos extender nuestros vectores a matrices (también se pueden ver los vectores como un caso especial de las matrices) agregando una dimensión tiempo. Por ejemplo, la actividad de las dos neuronas presinápticas a lo largo de tres instantes temporales puede especificarse como:

$$ \pmb{x} = \begin{bmatrix} 2 & 3 \\ 4 & 5 \\ 0 & 2 \end{bmatrix} $$

Dado el mismo vector de pesos sinápticos, ¿cómo podemos calcular la actividad de la neurona postsináptica a lo largo del tiempo? Una ventaja muy importante del álgebra lineal es que nos permite hacer estos cálculos haciendo exactamente la misma operación:

In [ ]:
x = np.array([[2, 3], [4, 5], [0, 2]])
w = np.array([1, 4])

print(x @ w)

## Plasticidad sináptica

La plasticidad sináptica es la capacidad de las conexiones entre neuronas (sinapsis) de cambiar su eficacia en función de la actividad. Dicho simple: el cerebro ajusta los pesos sinápticos según la experiencia, permitiendo aprender, recordar y adaptarse.

La plasticidad permite cambiar la probabilidad de que un potencial presináptico haga disparar a la neurona postsináptica. Lo puede lograr potenciando (LTP) o deprimiendo (LTD), y esos cambios pueden ser de corto o largo plazo. Hay muchas formas de que eso ocurra, una de ellas es debido al patrón temporal de disparos (p. ej., STDP: si la neurona presináptica dispara justo antes de la postsináptica, la sinapsis suele fortalecerse; si dispara después, suele debilitarse).

_Nota: Como veremos en el curso, en el contexto de modelos computacionales, "aprender" a menudo significa "ajustar pesos"._

Sin embargo, ¿cómo podemos ajustar los pesos para modelar la plasticidad? Una regla hebbiana básica se escribe como:

$$ \Delta w = \eta x_{pre} x_{post} $$

Ahora veremos una regla más sofisticada conocida como "Regla de Oja":

$$ \Delta w = \eta (x_{pre} x_{post} - x_{post}^2 w) $$

### Instanciación de las neuronas pre-sinápticas

En lugar de inicializar las matrices en forma manual, instanciaremos $x$ a lo largo de 100 saltos temporales en forma programática:

In [ ]:
n = 100
x1 = np.random.rand(n, 1)
x2 =  1 + 2 * x1 + 0.3 * np.random.rand(n, 1)

x = np.hstack([x1, x2])
w = np.array([-0.2, 0.5])

print(x @ w)

Recién imprimimos cual sería la actividad de la neurona post-sináptica si no hubiera plasticidad. Corré la siguiente celda para poder visualizar esta actividad modulada con diferentes pesos.

In [ ]:
@widgets.interact(w1=(-1, 1, 0.1), w2=(-1, 1, 0.1))
def simulate(w1 = -0.2, w2 = 0.5):
  w = np.array([w1, w2])
  visualizar_red(x, w)

## Simulación

Finalmente, iremos actualizando los pesos en cada salto temporal segpun la regla de Oja.

In [ ]:
# Valor inicial de los pesos sinápticos
w = np.array([-0.2, 0.5])

z = x - np.mean(x, axis=0)

# Graficamos los puntos
plt.scatter(z[:,0], z[:,1])

eta = 0.1

for i in range(n):
  # Actividad de la neurona pre-sináptica 1
  ri = z[i][0]

  # Actividad de la neurona pre-sináptica 2
  rj = z[i][1]

  # Actividad de la neurona post-sináptica con en este instante temporal
  rPost = ri * w[0] + rj * w[1]

  # Actualizamos los pesos según la regla de Oja
  w[0] = w[0] + eta * (rPost * ri - (rPost ** 2) * w[0])
  w[1] = w[1] + eta * (rPost * rj - (rPost ** 2) * w[1])

  # Graficamos una flecha con los pesos
  plt.arrow(0, 0, w[0].item(), w[1].item(), width=0.01, alpha=i/n)